In [5]:
import os
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from datetime import datetime
import logging
from imblearn.over_sampling import SMOTE

# Configuración
TEST_SIZE = 0.2
RANDOM_STATE = 42
MIN_SAMPLES_PER_CLASS = 25000  # Ajusta este valor según sea necesario

# Configurar logging
logging.basicConfig(
    filename='data_split.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def extract_metadata_from_filename(file_path):
    """
    Extrae metadatos del nombre del archivo y su ruta para mayor robustez.
    """
    # Obtener el nombre del archivo y la ruta completa
    file_name = os.path.basename(file_path)
    dir_path = os.path.dirname(file_path)

    # Patrón para archivos que terminan en _all_features.parquet
    all_features_pattern = r"""
        ^(?P<normalization>[A-Za-z]+)_  # Parte de normalización
        all_features\.parquet$          # Tipo de dataset
    """

    # Patrón para archivos que terminan en _selected.parquet
    selected_pattern = r"""
        ^(?P<normalization>[A-Za-z]+)_  # Parte de normalización
        (?P<selector>[A-Za-z-]+)_       # Selector de características
        selected\.parquet$              # Tipo de dataset
    """

    # Intentar coincidir con el patrón de "all_features"
    all_features_match = re.match(all_features_pattern, file_name, re.VERBOSE)
    if all_features_match:
        normalization = all_features_match.group("normalization")
        selector = "full"  # Asignar "full" como selector para estos archivos
        file_type = "full"
        return normalization, selector, file_type

    # Intentar coincidir con el patrón de "selected"
    selected_match = re.match(selected_pattern, file_name, re.VERBOSE)
    if selected_match:
        normalization = selected_match.group("normalization")
        selector = selected_match.group("selector")
        file_type = "selected"
        return normalization, selector, file_type

    # Si no coincide con ningún patrón, devolver None
    return None, None, None

def validate_data_structure(df):
    """Realiza validaciones completas de la estructura del DataFrame"""
    # Verificar columna target
    if 'nivel_triage' not in df.columns:
        raise ValueError("Columna target 'nivel_triage' no encontrada")
    
    # Verificar valores del target
    target_values = df['nivel_triage'].unique()
    if any(not isinstance(x, (int, np.integer)) for x in target_values):
        raise ValueError("Valores no enteros en la columna target")
    
    # Verificar distribución de clases
    class_distribution = df['nivel_triage'].value_counts()
    logging.info(f"Distribución de clases en el dataset: {class_distribution.to_dict()}")
    
    # Ajustar el umbral mínimo de muestras por clase
    if class_distribution.min() < MIN_SAMPLES_PER_CLASS:
        problematic = class_distribution[class_distribution < MIN_SAMPLES_PER_CLASS]
        raise ValueError(
            f"Clases con menos de {MIN_SAMPLES_PER_CLASS} muestras: {problematic.to_dict()}"
        )
    
    return True

def balance_classes(X, y):
    """Balancea las clases usando SMOTE"""
    smote = SMOTE(sampling_strategy='auto', k_neighbors=3, random_state=42)
    X_res, y_res = smote.fit_resample(X, y)
    return X_res, y_res

def safe_stratified_split(X, y):
    """Realiza split estratificado con manejo de clases pequeñas"""
    class_counts = y.value_counts()
    
    # Verificar si podemos estratificar
    can_stratify = all(count >= (1 / TEST_SIZE) for count in class_counts)
    
    if can_stratify:
        # Realizar split estratificado
        X_train, X_test, y_train, y_test = train_test_split(
            X, y,
            test_size=TEST_SIZE,
            random_state=RANDOM_STATE,
            stratify=y
        )
    else:
        logging.warning("Estratificación deshabilitada por clases muy pequeñas. Ajustando tamaño de prueba...")
        # Ajustar el tamaño de prueba para asegurar que todas las clases estén representadas
        min_samples = class_counts.min()
        test_size = min(1 / min_samples, TEST_SIZE)  # Asegurar al menos una muestra por clase en el test
        X_train, X_test, y_train, y_test = train_test_split(
            X, y,
            test_size=test_size,
            random_state=RANDOM_STATE,
            stratify=y
        )
    
    # Verificar que todas las clases estén presentes en ambos conjuntos
    train_classes = set(y_train.unique())
    test_classes = set(y_test.unique())
    
    if train_classes != test_classes:
        missing_train = test_classes - train_classes
        missing_test = train_classes - test_classes
        logging.warning(f"Clases faltantes en train: {missing_train}. Clases faltantes en test: {missing_test}. Ajustando split...")
        
        # Forzar presencia de todas las clases en train y test
        for cls in missing_train:
            idx = y_test[y_test == cls].index
            X_train = pd.concat([X_train, X_test.loc[idx]])
            y_train = pd.concat([y_train, y_test.loc[idx]])
            X_test = X_test.drop(idx)
            y_test = y_test.drop(idx)
        
        for cls in missing_test:
            idx = y_train[y_train == cls].index
            X_test = pd.concat([X_test, X_train.loc[idx]])
            y_test = pd.concat([y_test, y_train.loc[idx]])
            X_train = X_train.drop(idx)
            y_train = y_train.drop(idx)
    
    return X_train, X_test, y_train, y_test

def optimize_dtypes(df, target_column):
    """Optimiza los tipos de datos del DataFrame"""
    for col in df.columns:
        if col == target_column:
            if df[col].dtype != 'int8':
                df[col] = df[col].astype('int8')
        else:
            if df[col].dtype != 'float32':
                # Conservar solo las columnas con varianza
                if df[col].nunique() > 1:
                    df[col] = df[col].astype('float32')
                else:
                    df.drop(col, axis=1, inplace=True)
    return df

def create_train_test_split(input_file, output_base_path):
    """Crea splits train-test con validaciones mejoradas"""
    try:
        file_name = os.path.basename(input_file)
        logging.info(f"Iniciando procesamiento de: {file_name}")
        
        # Extraer metadatos usando la ruta completa
        norm_name, selector_name, file_type = extract_metadata_from_filename(input_file)
        if not all([norm_name, selector_name, file_type]):
            raise ValueError(f"Formato de nombre inválido: {file_name}")
        
        # Leer dataset
        if not os.path.exists(input_file):
            raise FileNotFoundError(f"Archivo no encontrado: {input_file}")
        
        df = pd.read_parquet(input_file)
        
        # Verificar el número de filas cargadas
        logging.info(f"Número de filas cargadas: {len(df)}")
        
        # Validar estructura
        validate_data_structure(df)
        
        # Optimizar tipos de datos
        df = optimize_dtypes(df, 'nivel_triage')
        
        # Preparar split
        X = df.drop(columns=['nivel_triage'])
        y = df['nivel_triage']
        
        # Balancear clases si es necesario
        X_res, y_res = balance_classes(X, y)
        
        # Split estratificado seguro
        X_train, X_test, y_train, y_test = safe_stratified_split(X_res, y_res)
        
        # Validar distribución post-split
        train_classes = set(y_train.unique())
        test_classes = set(y_test.unique())
        
        if train_classes != test_classes:
            missing = test_classes - train_classes
            logging.warning(f"Clases faltantes en train: {missing}. Ajustando split...")
            # Forzar presencia de todas las clases en train
            _, X_test, _, y_test = train_test_split(
                X_test, y_test,
                test_size=TEST_SIZE,
                random_state=RANDOM_STATE,
                stratify=y_test if len(y_test) > 0 else None
            )
        
        # Crear estructura de directorios
        file_id = f"{norm_name}_{selector_name}_{file_type}"
        output_path = os.path.join(output_base_path, file_id)
        os.makedirs(output_path, exist_ok=True)
        
        # Guardar archivos con compresión
        save_params = {
            'engine': 'pyarrow',
            'compression': 'gzip',
            'index': False
        }
        
        X_train.to_parquet(os.path.join(output_path, f"X_train_{file_id}.parquet"), **save_params)
        X_test.to_parquet(os.path.join(output_path, f"X_test_{file_id}.parquet"), **save_params)
        y_train.to_frame().to_parquet(os.path.join(output_path, f"y_train_{file_id}.parquet"), **save_params)
        y_test.to_frame().to_parquet(os.path.join(output_path, f"y_test_{file_id}.parquet"), **save_params)
        
        # Loggear métricas
        split_info = {
            'original_samples': len(df),
            'train_samples': len(X_train),
            'test_samples': len(X_test),
            'train_classes': y_train.nunique(),
            'test_classes': y_test.nunique(),
            'class_distribution_train': y_train.value_counts(normalize=True).to_dict(),
            'class_distribution_test': y_test.value_counts(normalize=True).to_dict()
        }
        logging.info(f"Split exitoso: {split_info}")
        
        print(f"✅ {file_name} procesado correctamente")
        return True
    
    except Exception as e:
        logging.error(f"Error procesando {file_name}: {str(e)}", exc_info=True)
        print(f"❌ Error en {file_name}: {str(e)}")
        return False

def main():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    input_root = r"C:\Users\Administrador\Documents\PythonScripts\Tesis\tesisaustral\outputs\experiments\experimento_final_20250223_181000"
    output_root = os.path.join(r"C:\Users\Administrador\Documents\PythonScripts\Tesis\tesisaustral\outputs\experiments", f"train_test_splits_{timestamp}")
    
    # Verificar la ruta de entrada
    print(f"Buscando archivos en: {input_root}")
    if not os.path.exists(input_root):
        raise FileNotFoundError(f"La ruta de entrada no existe: {input_root}")
    
    # Buscar recursivamente TODOS los archivos .parquet
    target_files = []
    for root, dirs, files in os.walk(input_root):
        print(f"Directorio actual: {root}")
        print(f"Subdirectorios: {dirs}")
        print(f"Archivos: {files}")
        for file in files:
            if file.endswith(".parquet"):
                full_path = os.path.join(root, file)
                target_files.append(full_path)
    
    print(f"\n🔍 Encontrados {len(target_files)} archivos Parquet:")
    for f in target_files:
        print(f"• {os.path.relpath(f, input_root)}")
    
    # Procesar con manejo de errores
    success_count = 0
    for idx, file_path in enumerate(target_files, 1):
        print(f"\n[{idx}/{len(target_files)}] Procesando: {os.path.basename(file_path)}")
        if create_train_test_split(file_path, output_root):
            success_count += 1
    
    # Reporte final
    print(f"\n🎉 Proceso completado: {success_count} éxitos, {len(target_files)-success_count} fallos")
    logging.info(f"Proceso finalizado: {success_count}/{len(target_files)} exitosos")

if __name__ == "__main__":
    main()

Buscando archivos en: C:\Users\Administrador\Documents\PythonScripts\Tesis\tesisaustral\outputs\experiments\experimento_final_20250223_181000
Directorio actual: C:\Users\Administrador\Documents\PythonScripts\Tesis\tesisaustral\outputs\experiments\experimento_final_20250223_181000
Subdirectorios: ['MaxAbs', 'MinMax', 'NoNorm', 'Robust', 'Standard']
Archivos: []
Directorio actual: C:\Users\Administrador\Documents\PythonScripts\Tesis\tesisaustral\outputs\experiments\experimento_final_20250223_181000\MaxAbs
Subdirectorios: []
Archivos: ['MaxAbs_all_features.parquet', 'MaxAbs_Linear_plot.png', 'MaxAbs_Linear_report.csv', 'MaxAbs_Linear_selected.parquet', 'MaxAbs_Model-Based_plot.png', 'MaxAbs_Model-Based_report.csv', 'MaxAbs_Model-Based_selected.parquet', 'MaxAbs_Nonlinear_plot.png', 'MaxAbs_Nonlinear_report.csv', 'MaxAbs_Nonlinear_selected.parquet']
Directorio actual: C:\Users\Administrador\Documents\PythonScripts\Tesis\tesisaustral\outputs\experiments\experimento_final_20250223_181000\Min